In [10]:
import pandas as pd
import re

df_nat = pd.read_csv("resultados_tarifas_naturgy.csv")
df_natsolar = pd.read_csv("resultados_tarifa_solar_naturgy.csv")
df_octopus = pd.read_csv("resultados_tarifas_octopus.csv")
df_repsol = pd.read_csv("resultados_tarifas_repsol_unificado.csv")
df_xenera = pd.read_csv("resultados_tarifas_xenera_unificado.csv")

# Potencia
#P1 Periodo Punta 
#P2 Periodo Llano
#P3 Periodo Valle


## OCTOPUS
df_octopus.columns = ['Tarifa', 'Descripción', 'Punta', 'Llano','Valle', 'P1', 'P3','Excedentes']
df_octopus['Empresa'] = "Octopus"
df_octopus.at[0, 'Llano'] = df_octopus.at[0, 'Punta']
df_octopus.at[0, 'Valle'] = df_octopus.at[0, 'Punta']

## REPSOL

df_repsol.columns = ['Tarifa', 'Descripción', 'Punta', 'P1','P3','Excedentes']
df_repsol['Llano'] = df_repsol['Punta']
df_repsol['Valle'] = df_repsol['Punta']
df_repsol.loc[2, 'Valle'] = 0.086415 # Durante 10 horas al día que se elijan.
df_repsol['Empresa'] = "Repsol"

## XENERA
df_xenera.columns = ['Valle', 'Llano', 'Punta', 'P1','Coste de gestión']
df_xenera['Llano'] = df_xenera['Llano'].fillna(df_xenera['Valle'])
df_xenera['Punta'] = df_xenera['Punta'].fillna(df_xenera['Valle'])
df_xenera['P1'] = 0.070357
df_xenera['P3'] = df_xenera['P1']
df_xenera['Tarifa'] = ["Home contigo","En casa","A vaquiña fija","Home contigo única"]
df_xenera['Empresa'] = "Xenera"

# Limpiamos nombres en Naturgy Solar

df_natsolar.columns = ['P1', 'P3', 'Punta', 'Llano','Valle', 'Excedentes','Tarifa']
df_natsolar['Empresa'] = "Naturgy"
df_natsolar['Tarifa'] = "Solar"

## Limpiamos nombres en Naturgy

df_naturgy = df_nat.transpose().reset_index()
df_naturgy.columns = ['Tarifa','Descripción', 'A', 'B','Punta','P1','C','D']
df_naturgy = df_naturgy.drop(columns=['A', 'B','C','D']).drop(1)
df_naturgy['Empresa'] = "Naturgy"
df_naturgy['P3'] = df_naturgy['P1']
df_naturgy['Llano'] = df_naturgy['Punta']
df_naturgy['Valle'] = df_naturgy['Punta']

# Corregimos las potencias

valor_punta_pot = df_naturgy['P1'].str.extract(r'Punta: ([\d,]+)')
valor_valle_pot = df_naturgy['P3'].str.extract(r'Valle: ([\d,]+)')
df_naturgy['P1'] = valor_punta_pot
df_naturgy['P3'] = valor_valle_pot
# Corregimos la fila 0

valor_inicial = re.search(r'([\d,]+)', df_naturgy.loc[0, 'Punta']).group(1)

# Asigna este valor a las columnas 'Punta', 'Llano' y 'Valle' en la fila 0
df_naturgy.at[0, 'Punta'] = valor_inicial if valor_inicial else None
df_naturgy.at[0, 'Llano'] = valor_inicial if valor_inicial else None
df_naturgy.at[0, 'Valle'] = valor_inicial if valor_inicial else None

# Corregimos la fila 1
fila_1 = df_naturgy.loc[2, 'Punta']
valor_punta = re.search(r'Punta: ([\d,]+€/kWh)', fila_1)
valor_llano = re.search(r'Llano: ([\d,]+€/kWh)', fila_1)
valor_valle = re.search(r'Valle: ([\d,]+€/kWh)', fila_1)
df_naturgy.at[2, 'Punta'] = valor_punta.group(1) if valor_punta else None
df_naturgy.at[2, 'Llano'] = valor_llano.group(1) if valor_llano else None
df_naturgy.at[2, 'Valle'] = valor_valle.group(1) if valor_valle else None

#Unimos todos los datos en una misma dataframe

df_companias = pd.concat([df_naturgy,df_natsolar,df_octopus,df_repsol,df_xenera], ignore_index=True)

# Añadimos el coste de la batería virtual de Repsol

df_companias['Bateria'] = df_companias['Excedentes']
df_companias.loc[2, 'Bateria'] = 0.0
df_companias.loc[6, 'Bateria'] = 0.0
df_companias.loc[10, 'Bateria'] = 1.80

# Cambiamos los NaN por 0

df_companias = df_companias.fillna(0)

# Reordenamos las columnas
orden = ['Empresa', 'Tarifa', 'Descripción', 'Punta', 'Llano',
       'Valle', 'P1', 'P3', 'Excedentes', 'Coste de gestión', 'Bateria']
df_companias=df_companias[orden]

def convertir_unidades(celda):
    if isinstance(celda, str) and '€/kWh' in celda:
        return float(celda.replace('€/kWh', '').replace(',', '.'))
    elif isinstance(celda, str) and '€/kW/día' in celda:
        return float(celda.replace('€/kW/día', '').replace(',', '.'))
    elif isinstance(celda, str) and '€/kW*día' in celda:
        return float(celda.replace('€/kW*día', '').replace(',', '.'))
    elif isinstance(celda, str) and '€ / mes' in celda:
        return float(celda.replace('€ / mes', '').replace(',', '.'))
    elif isinstance(celda, str) and ',' in celda:
        try:
            return float(celda.replace(',', '.'))
        except:
            pass
    return celda

    

# Aplica la función a todo el DataFrame
df_companias = df_companias.applymap(convertir_unidades)

#octopus = {
#    "nombre": "Octopus",
#    "punta": 0.1780, #€/kWh
#    "llano": 0.1300, #€/kWh
#    "valle": 0.0810, #€/kWh
#    "P1": 0.089, #€/kW/día
#    "P2": 0.026, #€/kW/día
#    "excedente": 0.05, #€/kWh
#    "batería": 0
#}
# Sacamos datos de las compañías sin tarifa solar

df_nosolar = df_companias[df_companias['Excedentes'] == 0].drop(['Excedentes', 'Bateria'], axis=1)

# Sacamos datos de las compañías con tarifa solar

df_solar = df_companias[df_companias['Excedentes'] > 0]

# Una vez limpitos, los metemos en un único data frame y lo guardamos en un csv
df = pd.DataFrame(df_companias)
dfs = pd.DataFrame(df_solar)
dfns = pd.DataFrame(df_nosolar)
df.to_csv("tarifas.csv", index=False)
dfs.to_csv("tarifas_solar.csv", index=False)
dfns.to_csv("tarifas_nosolar.csv", index=False)


# Verifica los cambios
df_companias

df_solar

,Empresa,Tarifa,Descripción,Punta,Llano,Valle,P1,P3,Excedentes,Coste de gestión,Bateria
2,Naturgy,Solar,0,0.170000,0.116200,0.082100,0.103663,0.034042,0.11,0.0,0.0
6,Octopus,Octopus Solar,Precio fijo para cada periodo,0.171000,0.123000,0.075000,0.089000,0.026000,0.05,0.0,0.0
10,Repsol,Tarifa Placa Solar,Precio de la energía y término fijo,0.139746,0.139746,0.139746,0.068219,0.068219,0.08,0.0,1.8
